In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import itertools
import time
import html5lib
import lxml

In [2]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

## Method to scrape the data(Comments) from the given source page

In [3]:
def Scrape(source, m1, m2, y1, y2):
    flag = 0
    month =  ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
    if m2<m1:
        flag = 1
    soup = BeautifulSoup(source,"html5lib")
    dates = soup.findAll('span' , class_="a-size-base a-color-secondary review-date")
    text = soup.findAll('span', class_="a-size-base review-text review-text-content")
    d = []
    for (date,element) in  zip(dates,text):
        temp = date.text.lower().split(" ")
        yr1 = temp[-1]
        yr = int(yr1)
        mon1 = temp[-2]
        if any (x in mon1 for x in month):
            mon1 = temp[-2]
        else:
            mon1 = temp[-3]
        mon = month.index(mon1)
        if((mon>=m1 and mon<=m2)and(yr>=y1 and yr<=y2) and flag==0)or(flag==1 and ((mon<=11 and mon>=m1)or(mon<=m2 and mon>=1))):
            t = element.text
            if "    "in t:
                t = t.split("    ")[1]
                t = t.split("\n")[0]
            else:
                t = t.split("\n")[0]
            d.append(t)
    return d

## Opening the web driver and accessing the Robocraze amazon page

In [4]:
loc = r"C:\Users\Sabarish\Downloads\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(loc)

home_link = "https://www.amazon.in/stores/roboCraze/roboCraze/page/19235B68-0603-400B-8CDE-71F79A088722"
driver.get("https://www.amazon.in/stores/roboCraze/roboCraze/page/19235B68-0603-400B-8CDE-71F79A088722")

driver.maximize_window()
time.sleep(2)

### If required by the user to enter the range of month and year to search comments

In [5]:
from_month , from_year  = input("Input from Month and year separated by a space: ").lower().split(" ")
to_month , to_year = input("Input to Month and year separated by a space: ").lower().split(" ")

month =  ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
m1 = month.index(from_month)
m2 = month.index(to_month)
y1 = int(from_year)
y2 = int(to_year)
if y2>y1:
    if m1<=m2:
        m1=0
        m2=11

Input from Month and year separated by a space: january 2020
Input to Month and year separated by a space: january 2023


## Selenium is used here to navigate into each category -> product -> comments

In [6]:
lnks = driver.find_elements(By.CLASS_NAME, 'EditorialTile__innerContent__n92i8')
len(lnks)
#lnks[3].click()

14

In [7]:
data = []

for i in range(1,12): 
# 1-13 as the website has 14 clickable items with 2 of it not being any category

    time.sleep(3)
    lnks = driver.find_elements(By.CLASS_NAME, 'EditorialTile__innerContent__n92i8')
    lnks.pop(11)
    head = lnks[i].text
    print(head)
    lnks[i].click()
    temp_url = driver.current_url
    lnks1 = driver.find_elements(By.XPATH, '//li[@class="ProductGridItem__itemOuter__KUtvv ProductGridItem__fixed__DQzmO"]')
    x = len(lnks1)
    print(x)
    flag = 1
    for j in range(0,x):
        lnks1 = driver.find_elements(By.XPATH, '//li[@class="ProductGridItem__itemOuter__KUtvv ProductGridItem__fixed__DQzmO"]')
        lnks1[j].click()
        try:
            link2 = driver.find_element(By.XPATH, '//a[@class="a-link-emphasis a-text-bold"]')
            link2.click()
            data = data + (Scrape(driver.page_source, m1, m2, y1, y2))
            c = 0
            while True:
                try:
                    time.sleep(2)
#                     driver.navigate().refresh()
                    if driver.find_element(By.XPATH, '//li[@class="a-last"]'):
                        nxt_pg = driver.find_element(By.XPATH, '//li[@class="a-last"]')
                        nxt_pg.click()
                        data = data + (Scrape(driver.page_source, m1, m2, y1, y2))
                except NoSuchElementException:
                    driver.get(temp_url)
                    flag = 0
                    break
        except NoSuchElementException:
            driver.get(temp_url)
            time.sleep(2)
            flag = 0
        flag = 1
    driver.get(home_link)

QUADCOPTER
24
DIY
24
RASPBERRY PI
8
CONTROLLERS
24
SENSORS
24
INTERNET OF THINGS
8
Arduino
10
Ebike
3
3D Printer and Accessories
21
Power Regulator
13


IndexError: pop index out of range

In [8]:
data

['It’s good that they have the return/replacement policy. I had to return it because one of the channels wasn’t working, i had checked on t6config as well, channel 2 wasn’t responding. The replacement came quick and now everything works. But still I would recommend to buy the one that has screen on it because the software they provide is really old and doesn’t work with newer version of windows. I couldn’t find any working software for my mac as well, so i had fond a win8 system to configure it.',
 'Product is being shelled at high price if you want you can buy it from robu.in',
 'First time similar item received in damaged condition.  Item was return and timely refund received.This time item was received in good condition, but Data cable, set-up CD &  manual were missing.Kindly arrange to deliver the missing items.Regards',
 'Easy to use...nice product',
 "Overall the product is good and the quality which is used is good but the cd which come along is blank and i couldn't able to inst

In [9]:
len(data)

3449

### Converting the Scraped data list into a dataframe and storing in an excel

In [10]:
df = pd.DataFrame()
df["Comments"] = data
df.head

<bound method NDFrame.head of                                                Comments
0     It’s good that they have the return/replacemen...
1     Product is being shelled at high price if you ...
2     First time similar item received in damaged co...
3                            Easy to use...nice product
4     Overall the product is good and the quality wh...
...                                                 ...
3444  Not working... Voltage output is very less and...
3445  Does exactly what it is supposed to do. Used i...
3446  I need to lower the RPM for My metal grinderSo...
3447                       product quality is very good
3448  Very good product. My centrifuge machine was n...

[3449 rows x 1 columns]>

In [11]:
df.to_excel("Comments.xlsx")
df

,Comments
0,It’s good that they have the return/replacemen...
1,Product is being shelled at high price if you ...
2,First time similar item received in damaged co...
3,Easy to use...nice product
4,Overall the product is good and the quality wh...
...,...
3444,Not working... Voltage output is very less and...
3445,Does exactly what it is supposed to do. Used i...
3446,I need to lower the RPM for My metal grinderSo...
3447,product quality is very good
